In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
from datetime import datetime
import time
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from binance.client import Client
from binance.spot import Spot
from crypto.utils import date_to_milliseconds, int_to_date
from crypto.get_dca import get_historical_price
from crypto.get_dca import DCA


In [ ]:
# spark = SparkSession.builder.getOrCreate()
# sc = spark.sparkContext

In [ ]:
# binanceAPI1
KEY = 'b25TtEWHvfDybaa0mMfkJ9CY4HCvP4sDBWj8no3mVvbD6bTLVjPyUo25tSFOGrgT'
SECRET = 'EFtFBJblJUJZ4WW16f3yJmXxnFVah0YObA8Sm0z7mCcNOQO1EBqDO8b1HLY8KIuM'
BASE_URL = 'https://api.binance.com' # production base url
# BASE_URL = 'https://testnet.binance.vision' # testnet base url

client = Client(KEY, SECRET)



1. first figure out how to get average buying price

2. second recursively apply buying price to portfolio

empezemos

lets start off by taking a par e.g sol/usdt

we will figure out the equivalent in various currencies e.g usd, eur, btc, bnb

In [ ]:
exchange_info = client.get_exchange_info()

In [ ]:
d = DCA()

In [ ]:
a = ['KSM', 'SOL']

In [ ]:
list(
    filter(
        lambda x: x.endswith(tuple(a))
        or x.startswith(tuple(a)), d.full_pair_list))

In [ ]:
filter_by_currency(d.full_pair_list, 'KSM')

In [ ]:
set([e['baseAsset'] for e in exchange_info['symbols']]).intersection(
    set([e['symbol'] for e in exchange_info['symbols']]))


In [ ]:
set([e['baseAsset'] for e in exchange_info['symbols']])

In [ ]:
# get all corresponding pairs
full_coin_list = list(set([e['symbol'] for e in exchange_info['symbols']]))
coin = 'BTC'
target_currency = 'USDT'
pairs = list(filter(lambda x: x.__contains__(coin), full_coin_list))
len(pairs)


In [ ]:
# get all orders for those pairs
pair_orders = []
idx=0
for i in pairs:
    # print(i)
    idx += 1
    pair_orders.extend(client.get_all_orders(symbol=i, limit=1))
    if idx % 3 == 0:
        time.sleep(1)


In [ ]:
# get currency list (currency coin was bought in)
coin_set = set([o['symbol'] for o in pair_orders])
print(coin_set)
current_currency_list = list(map(lambda p: p.replace(coin, ''), coin_set))

current_currency_list

In [ ]:
def filter_by_currency(inp, currency_filter):
    # cannot use 'contains' e.g. 'CKBUSDT' contains BUSD but is not BUSD
    return list(filter(lambda x: x.endswith(currency_filter) or x.startswith(currency_filter), inp))

    

In [ ]:
# get current to target currency pair and check 1 occurance
final_convertion_dict = {}
target_currency_list = list(
    filter(lambda e: e.__contains__(target_currency), full_coin_list))
for c in current_currency_list:
    if c==target_currency:
        final_convertion_dict[c]=c
    else:
        print(c)
        # find single currency - test possibilities
        # e.g. usdtbusd or busdusdt
        _target_pair = c + target_currency
        position = 1
        filtered = filter_by_currency(target_currency_list, _target_pair)
        if len(filtered)==0:
            _target_pair = target_currency + c
            position = 2
            filtered = filter_by_currency(target_currency_list, _target_pair)
        assert len(filtered)==1
        final_convertion_dict[c] = filtered[0]
final_convertion_dict

In [ ]:
currency_target = {}
for order in pair_orders:
    if order['status'] != 'CANCELED':
        _symbol = order['symbol'].replace(coin, '')
        if _symbol==target_currency:
            order['conversion_pair'] = 'NA'
            order['target_currency_price'] = 1
            order['target_price'] = float(order['price'])
        else:
            print(final_convertion_dict[_symbol], order['updateTime'])
            order['conversion_pair'] = final_convertion_dict[_symbol]
            order['target_currency_price'] = get_historical_price(client, final_convertion_dict[_symbol], order['updateTime'])['price']
            if position==1:
                order['target_price'] = float(order['price'])/float(order['target_currency_price'])
            else:
                order['target_price'] = float(order['price'])*float(order['target_currency_price'])



In [ ]:
pair_orders

In [ ]:
import json
with open(f'data_{time.strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
    json.dump(pair_orders, outfile)

In [ ]:
def filter_by_currency(inp, currency_filter):
        # cannot use 'contains' e.g. 'CKBUSDT' contains BUSD but is not BUSD
        return list(
            filter(
                lambda x: x.endswith(tuple(currency_filter))
                or x.startswith(tuple(currency_filter)), inp))

In [ ]:
current_currency_list

In [ ]:
a = None

In [ ]:
b = a or 2

In [ ]:
b